In [46]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [47]:
import numpy as np
import pandas as pd
from si.data import Dataset
import os

DIR = os.path.dirname(os.path.realpath('.'))
filename = os.path.join(DIR, 'datasets/rnn.data')
data = pd.read_csv(filename, names=["text", "class"])
dataset = Dataset.from_dataframe(data, ylabel="class")

def create_inputs(text, vocab_size, word_to_idx, idx_to_word, sentence_size):
    words = []
    for w in text[0].split(' '):
        v = np.zeros((vocab_size, 1))
        v[word_to_idx[w]] = 1
        words.append(v)
    return np.hstack([np.hstack(words), np.zeros((vocab_size, sentence_size-len(words)))]).T


def encode_dataset(dataset):
    inputs = []
    # Create the vocabulary.
    vocab = list(set([w for text in dataset.X for w in text[0].split(' ')]))
    vocab_size = len(vocab)
    sentence_size = max([len(text[0].split(' ')) for text in dataset.X])
    print('%d unique words found' % vocab_size)

    # Assign indices to each word.
    word_to_idx = { w: i for i, w in enumerate(vocab) }
    idx_to_word = { i: w for i, w in enumerate(vocab) }
    for text in dataset.X:
        inputs.append(np.array(create_inputs(text, vocab_size, word_to_idx, idx_to_word, sentence_size)))
    dataset.X = np.stack(inputs, axis=0)
    return  dataset, vocab, vocab_size, word_to_idx, idx_to_word

dataset, vocab, vocab_size, word_to_idx, idx_to_word = encode_dataset(dataset)

18 unique words found


In [51]:
from si.util.metrics import cross_entropy, cross_entropy_prime
from si.supervised import NN, Dense, Activation, Conv2D, Flatten, MaxPooling2D, RNN
from si.util.activation import Tanh, Sigmoid

net = NN(epochs=100,lr=0.1,verbose=False)
net.add(RNN(vocab_size,2))
net.add(Activation(Sigmoid()))
net.use_loss(cross_entropy, cross_entropy_prime)

net.fit(dataset)


ValueError: operands could not be broadcast together with shapes (2,10) (58,) 